In [1]:
import pandas as pd
import numpy as np

In [3]:
original_df = pd.read_csv('../04 - Data/Raw_Data/vHoneyNeonic_v02.csv')
original_df.head()

,state,numcol,yieldpercol,totalprod,stocks,priceperlb,prodvalue,year,StateName,Region,nCLOTHIANIDIN,nIMIDACLOPRID,nTHIAMETHOXAM,nACETAMIPRID,nTHIACLOPRID,nAllNeonic
0,AL,11000.0,56,616000.0,209000.0,1.49,918000.0,2007,Alabama,South,7696.2,3258.1,4149.6,0.0,0.0,15103.9
1,AL,11000.0,72,792000.0,230000.0,1.21,958000.0,2006,Alabama,South,680.0,4230.2,5371.6,0.0,0.0,10281.8
2,AL,12000.0,86,1032000.0,103000.0,1.18,1218000.0,2002,Alabama,South,0.0,1178.8,840.4,0.0,0.0,2019.2
3,AL,12000.0,87,1044000.0,282000.0,1.41,1472000.0,2004,Alabama,South,2676.4,1323.9,3863.9,0.0,0.0,7864.2
4,AL,13000.0,66,858000.0,266000.0,1.02,875000.0,2005,Alabama,South,1503.6,994.5,5493.9,0.0,0.0,7992.0


In [4]:
####### NORTH DAKOTA'S PREVIOUS YIELD #######

# Get the North Dakota's yield per colony by year
nd_df = original_df[original_df['StateName'] == 'North Dakota']
nd_df = nd_df[['year', 'yieldpercol']]
nd_df['Next Year'] = nd_df['year'] + 1
nd_df.rename(columns={'yieldpercol':'NorthDakota_PrevYear'}, inplace=True)

In [5]:
# Merge ND prev year yield
df = original_df.copy()
df = original_df.merge(nd_df[['Next Year', 'NorthDakota_PrevYear']], 
                       left_on='year', right_on='Next Year', how='left')
df.drop(columns='Next Year', inplace=True)

In [6]:
####### PREVIOUS YIELD BY STATE #######

prev_yield_df = original_df[['StateName', 'year', 'yieldpercol']]
prev_yield_df['NextYear'] = prev_yield_df['year'] + 1
prev_yield_df.rename(columns={'yieldpercol':'prevyearyield'}, inplace=True)

C:\Users\kalat\AppData\Local\Temp\ipykernel_9612\3406972834.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_yield_df['NextYear'] = prev_yield_df['year'] + 1
C:\Users\kalat\AppData\Local\Temp\ipykernel_9612\3406972834.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_yield_df.rename(columns={'yieldpercol':'prevyearyield'}, inplace=True)


In [7]:
# Join previous year's yield
interim_df = df.merge(prev_yield_df[['NextYear', 'prevyearyield', 'StateName']], left_on=['year', 'StateName'], right_on=['NextYear', 'StateName'], how='left')
interim_df.drop(columns='NextYear', inplace=True)
df = interim_df.copy()

In [8]:
df.columns

Index(['state', 'numcol', 'yieldpercol', 'totalprod', 'stocks', 'priceperlb',
       'prodvalue', 'year', 'StateName', 'Region', 'nCLOTHIANIDIN',
       'nIMIDACLOPRID', 'nTHIAMETHOXAM', 'nACETAMIPRID', 'nTHIACLOPRID',
       'nAllNeonic', 'NorthDakota_PrevYear', 'prevyearyield'],
      dtype='object')

In [9]:
####### PREVIOUS PESTICIDE DEPLOYEMENTS BY STATE YEAR #######
prev_pest_df = df[['StateName', 'year', 
                   'nCLOTHIANIDIN','nIMIDACLOPRID', 'nTHIAMETHOXAM', 
                   'nACETAMIPRID', 'nTHIACLOPRID']]
prev_pest_df['NextYear'] = prev_pest_df['year'] + 1

# Make renaming dict
rename_dict = {}
for col in prev_pest_df.columns[2:-1]:
    rename_dict[col] = 'prevyear_' + col

prev_pest_df.rename(columns=rename_dict, inplace=True)
prev_pest_df.drop(columns='year', inplace=True)

C:\Users\kalat\AppData\Local\Temp\ipykernel_9612\146946584.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_pest_df['NextYear'] = prev_pest_df['year'] + 1
C:\Users\kalat\AppData\Local\Temp\ipykernel_9612\146946584.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_pest_df.rename(columns=rename_dict, inplace=True)
C:\Users\kalat\AppData\Local\Temp\ipykernel_9612\146946584.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [10]:
interim_df = df.copy()
interim_df = interim_df.merge(prev_pest_df,
                 left_on=['StateName', 'year'], right_on=['StateName', 'NextYear'], how='left')
interim_df.drop(columns='NextYear', inplace=True)
df = interim_df.copy()

In [11]:
######## FILL NULLS #########

# Fill prevyearyield with state average
state_avg_yield = df[['StateName', 'yieldpercol']]
state_avg_yield = state_avg_yield.groupby('StateName').mean()
state_avg_yield.reset_index(inplace=True)
state_avg_yield.rename(columns={'yieldpercol':'avgyield'}, inplace=True)

interim_df = df.copy()
interim_df = interim_df.merge(right=state_avg_yield, on='StateName', how='left')
interim_df.prevyearyield = np.where(interim_df.prevyearyield.isnull(), interim_df.avgyield, interim_df.prevyearyield)
interim_df.drop(columns='avgyield', inplace=True)
df = interim_df.copy()

In [12]:
# North Dakota Previous Year Nulls
nd_avg_yield = np.mean(df[df['StateName'] == 'North Dakota']['yieldpercol'])
interim_df = df.copy()
interim_df['NorthDakota_PrevYear'].fillna(value=nd_avg_yield, inplace=True)
df = interim_df.copy()

In [13]:
nCOLS = list(df.columns[10:16])
prevyear_nCOLS = list(df.columns[18:])
all_cols_list = nCOLS + prevyear_nCOLS
states = set(list(df['StateName']))
interim_df = df.copy()

for col in all_cols_list:
    for state in states:
        state_nCOL_avg = np.mean(interim_df[interim_df['StateName'] == state][str(col)])
        interim_df[str(col)].fillna(value=state_nCOL_avg, inplace=True)
df = interim_df.copy()

In [14]:
df.columns
df_col_list = ['StateName', 'year', 'numcol', 'yieldpercol', 'totalprod', 'stocks', 'priceperlb',
       'prodvalue', 'Region', 'nCLOTHIANIDIN',
       'nIMIDACLOPRID', 'nTHIAMETHOXAM', 'nACETAMIPRID', 
       'nTHIACLOPRID', 'nAllNeonic',
       'prevyear_nCLOTHIANIDIN', 'prevyear_nIMIDACLOPRID',
       'prevyear_nTHIAMETHOXAM', 'prevyear_nACETAMIPRID',
       'prevyear_nTHIACLOPRID',  'NorthDakota_PrevYear', 'prevyearyield']
df = df[df_col_list]

In [16]:
df.to_csv('../04 - Data/Final Data/honey_neonic_cleaned.csv')

In [17]:
pd.get_dummies(df).to_csv('../04 - Data/Final Data/honey_neonic_cleaned_dummies.csv')